# 배터리 데이터 분석 워크플로우

이 노트북은 실제 배터리 테스트 데이터를 로드하고 전체 분석 워크플로우를 수행합니다.

## 워크플로우

1. **데이터 로딩**: PNE/Toyo 충방전기 데이터 자동 로드
2. **데이터 탐색**: 사이클 데이터 요약 및 시각화
3. **프로파일 분석**: 전압-용량 곡선 분석
4. **용량 열화 모델링**: 경험적 모델 피팅
5. **수명 예측**: EOL 사이클 및 승인 수명 예측
6. **리포트 생성**: 분석 결과 요약

---

## 0. 환경 설정

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# battery_analysis_v2 모듈 import
from battery_analysis_v2.core.data_loader import (
    detect_cycler_type,
    PNELoader,
    ToyoLoader,
    CyclerType
)

from battery_analysis_v2.core.life_prediction.empirical import (
    CapacityDegradationModel,
    ApprovalLifePredictor,
    ModelParameters
)

# 그래프 스타일 설정
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print('✓ 모듈 import 완료')

## 1. 데이터 로딩

### 1.1 데이터 경로 설정

**아래 셀에서 실제 배터리 테스트 데이터 경로를 입력하세요.**

In [ ]:
# ==========================================
# 데이터 경로 입력 (사용자 수정 필요)
# ==========================================

# 예제 경로:
# PNE: r"C:\Data\Battery\M01Ch001[001]"
# Toyo: r"C:\Data\Battery\channel_01"

DATA_PATH = r"C:\path\to\your\battery\data"  # 👈 여기를 실제 경로로 수정하세요

# 정격 용량 (mAh) - None이면 자동 추정
RATED_CAPACITY = None  # 예: 4500.0

# 시험 조건
TEST_TEMPERATURE_C = 45  # 시험 온도 (°C)
TARGET_TEMPERATURE_C = 25  # 실사용 목표 온도 (°C)

print(f"데이터 경로: {DATA_PATH}")
print(f"정격 용량: {RATED_CAPACITY if RATED_CAPACITY else '자동 추정'}")
print(f"시험 온도: {TEST_TEMPERATURE_C}°C")
print(f"목표 온도: {TARGET_TEMPERATURE_C}°C")

### 1.2 충방전기 타입 자동 감지

In [ ]:
# 충방전기 타입 감지
cycler_type = detect_cycler_type(DATA_PATH)

print(f"\n=== 충방전기 타입 감지 ===")
print(f"경로: {DATA_PATH}")
print(f"타입: {cycler_type.name}")

# 적절한 로더 생성
if cycler_type == CyclerType.PNE:
    loader = PNELoader(DATA_PATH, rated_capacity=RATED_CAPACITY)
    print("✓ PNELoader 생성 완료")
elif cycler_type == CyclerType.TOYO:
    loader = ToyoLoader(DATA_PATH, rated_capacity=RATED_CAPACITY)
    print("✓ ToyoLoader 생성 완료")
else:
    raise ValueError(f"알 수 없는 충방전기 타입: {cycler_type}")

print(f"\n정격 용량: {loader.rated_capacity:.1f} mAh")

### 1.3 사이클 데이터 로드

In [ ]:
# 사이클 데이터 로드
print("사이클 데이터 로딩 중...")
cycle_data = loader.load_cycle_data()

# DataFrame으로 변환
df_cycle = cycle_data.to_dataframe()

print(f"\n=== 사이클 데이터 로드 완료 ===")
print(f"총 사이클 수: {len(df_cycle)}")
print(f"사이클 범위: {df_cycle['cycle'].min()} ~ {df_cycle['cycle'].max()}")
print(f"\n첫 5개 사이클:")
display(df_cycle.head())

## 2. 데이터 탐색 및 시각화

### 2.1 기본 통계

In [ ]:
# 기본 통계 정보
print("=== 사이클 데이터 요약 통계 ===")
print(f"\n데이터 포인트: {len(df_cycle)}개")
print(f"\n주요 컬럼:")
for col in ['discharge_capacity', 'charge_capacity', 'efficiency', 'soh']:
    if col in df_cycle.columns:
        print(f"  {col}:")
        print(f"    평균: {df_cycle[col].mean():.4f}")
        print(f"    최소: {df_cycle[col].min():.4f}")
        print(f"    최대: {df_cycle[col].max():.4f}")
        print()

# 용량 유지율 계산
retention = cycle_data.capacity_retention()
print(f"\n용량 유지율:")
print(f"  초기: 100.0%")
print(f"  현재 (사이클 {df_cycle['cycle'].max()}): {retention[-1]*100:.2f}%")
print(f"  감소량: {(1-retention[-1])*100:.2f}%")

### 2.2 용량 트렌드 시각화

In [ ]:
# 용량 트렌드 그래프
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# 1. 방전 용량
ax = axes[0, 0]
ax.plot(df_cycle['cycle'], df_cycle['discharge_capacity'], 'b-', linewidth=1.5, marker='o', markersize=3)
ax.set_xlabel('사이클', fontsize=11)
ax.set_ylabel('방전 용량 (mAh)', fontsize=11)
ax.set_title('방전 용량 vs 사이클', fontsize=13, fontweight='bold')
ax.axhline(y=loader.rated_capacity * 0.8, color='r', linestyle='--', alpha=0.7, label='EOL (80%)')
ax.legend()

# 2. 용량 유지율
ax = axes[0, 1]
retention_pct = retention * 100
ax.plot(df_cycle['cycle'], retention_pct, 'g-', linewidth=1.5, marker='o', markersize=3)
ax.set_xlabel('사이클', fontsize=11)
ax.set_ylabel('용량 유지율 (%)', fontsize=11)
ax.set_title('용량 유지율 vs 사이클', fontsize=13, fontweight='bold')
ax.axhline(y=80, color='r', linestyle='--', alpha=0.7, label='EOL (80%)')
ax.set_ylim(70, 105)
ax.legend()

# 3. 효율
ax = axes[1, 0]
if 'efficiency' in df_cycle.columns:
    ax.plot(df_cycle['cycle'], df_cycle['efficiency'], 'orange', linewidth=1.5, marker='o', markersize=3)
    ax.set_xlabel('사이클', fontsize=11)
    ax.set_ylabel('효율 (%)', fontsize=11)
    ax.set_title('효율 vs 사이클', fontsize=13, fontweight='bold')
    ax.set_ylim(90, 102)
else:
    ax.text(0.5, 0.5, '효율 데이터 없음', ha='center', va='center', fontsize=14)
    ax.axis('off')

# 4. SOH
ax = axes[1, 1]
if 'soh' in df_cycle.columns:
    ax.plot(df_cycle['cycle'], df_cycle['soh'], 'purple', linewidth=1.5, marker='o', markersize=3)
    ax.set_xlabel('사이클', fontsize=11)
    ax.set_ylabel('SOH (%)', fontsize=11)
    ax.set_title('SOH vs 사이클', fontsize=13, fontweight='bold')
    ax.axhline(y=80, color='r', linestyle='--', alpha=0.7, label='EOL (80%)')
    ax.legend()
else:
    # SOH가 없으면 계산
    soh = cycle_data.soh() * 100
    ax.plot(df_cycle['cycle'], soh, 'purple', linewidth=1.5, marker='o', markersize=3)
    ax.set_xlabel('사이클', fontsize=11)
    ax.set_ylabel('SOH (%)', fontsize=11)
    ax.set_title('SOH vs 사이클', fontsize=13, fontweight='bold')
    ax.axhline(y=80, color='r', linestyle='--', alpha=0.7, label='EOL (80%)')
    ax.legend()

plt.tight_layout()
plt.show()

print("✓ 용량 트렌드 시각화 완료")

## 3. 프로파일 데이터 분석

### 3.1 특정 사이클 프로파일 로드

In [ ]:
# 분석할 사이클 선택 (첫 사이클, 중간, 마지막)
cycles_to_analyze = [
    df_cycle['cycle'].min(),
    df_cycle['cycle'].iloc[len(df_cycle)//2],
    df_cycle['cycle'].max()
]

print(f"분석할 사이클: {cycles_to_analyze}")

# 프로파일 데이터 로드
profiles = {}
for cycle_num in cycles_to_analyze:
    try:
        profile = loader.load_profile_data(int(cycle_num))
        profiles[cycle_num] = profile
        print(f"✓ 사이클 {cycle_num} 프로파일 로드 완료 (데이터 포인트: {len(profile.time)})")
    except Exception as e:
        print(f"✗ 사이클 {cycle_num} 로드 실패: {e}")

### 3.2 전압-용량 곡선 시각화

In [ ]:
# 전압-용량 곡선
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors = ['blue', 'green', 'red']
labels = ['초기', '중간', '최종']

# 왼쪽: 전체 프로파일
ax = axes[0]
for (cycle_num, profile), color, label in zip(profiles.items(), colors, labels):
    ax.plot(profile.capacity, profile.voltage, color=color, linewidth=2, 
            label=f'{label} (사이클 {cycle_num})', alpha=0.7)
ax.set_xlabel('용량 (mAh)', fontsize=12)
ax.set_ylabel('전압 (V)', fontsize=12)
ax.set_title('전압-용량 곡선 (전체)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# 오른쪽: 방전 프로파일만
ax = axes[1]
for (cycle_num, profile), color, label in zip(profiles.items(), colors, labels):
    # 방전 구간 필터링 (condition == 2)
    if profile.condition is not None:
        discharge_profile = profile.filter_by_condition(2)
        ax.plot(discharge_profile.capacity, discharge_profile.voltage, 
                color=color, linewidth=2, label=f'{label} (사이클 {cycle_num})', alpha=0.7)
    else:
        # condition 정보가 없으면 전체 표시
        ax.plot(profile.capacity, profile.voltage, color=color, linewidth=2, 
                label=f'{label} (사이클 {cycle_num})', alpha=0.7)

ax.set_xlabel('용량 (mAh)', fontsize=12)
ax.set_ylabel('전압 (V)', fontsize=12)
ax.set_title('방전 전압-용량 곡선', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ 전압-용량 곡선 시각화 완료")

## 4. 용량 열화 모델링

### 4.1 데이터 준비

In [ ]:
# 모델링을 위한 데이터 준비
# 사이클 수
cycles = df_cycle['cycle'].values

# 온도 (K 단위로 변환)
temperatures = np.full_like(cycles, 273.15 + TEST_TEMPERATURE_C, dtype=float)

# 정규화된 용량 (0-1 범위)
initial_capacity = df_cycle['discharge_capacity'].iloc[0]
capacities = df_cycle['discharge_capacity'].values / initial_capacity

print("=== 모델링 데이터 준비 ===")
print(f"사이클 수: {len(cycles)}")
print(f"사이클 범위: {cycles.min()} ~ {cycles.max()}")
print(f"온도: {TEST_TEMPERATURE_C}°C ({temperatures[0]:.2f}K)")
print(f"용량 범위: {capacities.min():.4f} ~ {capacities.max():.4f}")
print(f"초기 용량: {initial_capacity:.2f} mAh")

### 4.2 모델 피팅

In [ ]:
# 용량 열화 모델 생성 및 피팅
print("용량 열화 모델 피팅 중...")
model = CapacityDegradationModel()

try:
    result = model.fit(cycles, temperatures, capacities, maxfev=100000)
    
    print("\n=== 피팅 결과 ===")
    print(f"RMSE: {result.rmse:.6f}")
    print(f"R²: {result.r_squared:.6f}")
    print("\n최적 파라미터:")
    params_dict = result.to_dict()
    for key in ['a', 'b', 'b1', 'c', 'd', 'e', 'f', 'fd']:
        print(f"  {key:3s}: {params_dict[key]:12.6f}")
    
    fitting_success = True
except Exception as e:
    print(f"\n✗ 피팅 실패: {e}")
    print("기본 파라미터를 사용합니다.")
    fitting_success = False

### 4.3 피팅 결과 시각화

In [ ]:
if fitting_success:
    # 예측값 생성
    predict_cycles = np.linspace(cycles.min(), cycles.max() * 1.5, 300)
    predict_temps = np.full_like(predict_cycles, 273.15 + TEST_TEMPERATURE_C)
    predicted_capacity = model.predict(predict_cycles, predict_temps)
    
    # 시각화
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 왼쪽: 피팅 결과
    ax = axes[0]
    ax.scatter(cycles, capacities, alpha=0.6, s=50, label='실험 데이터', color='blue')
    ax.plot(predict_cycles, predicted_capacity, 'r-', linewidth=2, label='피팅된 모델')
    ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.7, label='EOL (80%)')
    ax.set_xlabel('사이클', fontsize=12)
    ax.set_ylabel('잔존 용량 비율', fontsize=12)
    ax.set_title(f'용량 열화 모델 피팅 ({TEST_TEMPERATURE_C}°C)', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    # 오른쪽: 잔차
    ax = axes[1]
    fitted_values = model.predict(cycles, temperatures)
    residuals = capacities - fitted_values
    ax.scatter(cycles, residuals, alpha=0.6, s=50, color='blue')
    ax.axhline(y=0, color='r', linestyle='--', alpha=0.7)
    ax.set_xlabel('사이클', fontsize=12)
    ax.set_ylabel('잔차', fontsize=12)
    ax.set_title('잔차 플롯', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # 통계 정보 추가
    textstr = f'RMSE = {result.rmse:.4f}\nR² = {result.r_squared:.4f}'
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=11,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    print("✓ 피팅 결과 시각화 완료")
else:
    print("피팅 실패로 시각화를 건너뜁니다.")

## 5. 수명 예측

### 5.1 시험 조건에서의 EOL 예측

In [ ]:
if fitting_success:
    # EOL 사이클 예측 (80% 용량 기준)
    eol_cycle = model.predict_cycle_to_eol(
        temperature=273.15 + TEST_TEMPERATURE_C,
        eol_threshold=0.8,
        max_cycles=10000
    )
    
    print("=== EOL 예측 (시험 조건) ===")
    print(f"시험 온도: {TEST_TEMPERATURE_C}°C")
    print(f"EOL 기준: 80% 잔존 용량")
    print(f"예측 EOL 사이클: {eol_cycle:,} 사이클")
    
    # EOL 시점의 용량 확인
    eol_capacity = model.predict(
        np.array([eol_cycle]), 
        np.array([273.15 + TEST_TEMPERATURE_C])
    )[0]
    print(f"EOL 시점 예측 용량: {eol_capacity:.4f}")
else:
    print("모델 피팅 실패로 EOL 예측을 건너뜁니다.")
    eol_cycle = None

### 5.2 승인 수명 예측 (Arrhenius 외삽)

In [ ]:
if fitting_success:
    # 승인 수명 예측기 생성
    predictor = ApprovalLifePredictor(
        test_temperature=273.15 + TEST_TEMPERATURE_C,
        target_temperature=273.15 + TARGET_TEMPERATURE_C,
        cycles_per_year=365,  # 하루 1사이클
        required_years=8.0
    )
    
    print("\n승인 수명 예측 중...")
    
    # 피팅 및 예측
    approval_result = predictor.fit_and_predict(
        cycles=cycles,
        temperatures=temperatures,
        capacities=capacities,
        eol_threshold=0.8,
        activation_energy=50000  # 50 kJ/mol
    )
    
    print("\n=== 승인 수명 예측 결과 ===")
    print(f"시험 온도: {TEST_TEMPERATURE_C}°C")
    print(f"실사용 온도: {TARGET_TEMPERATURE_C}°C")
    print(f"활성화 에너지: 50 kJ/mol")
    print()
    print(f"시험 EOL 사이클: {approval_result.test_cycles:,} 사이클")
    print(f"가속 계수: {approval_result.acceleration_factor:.2f}배")
    print(f"실사용 환산 사이클: {approval_result.equivalent_real_cycles:,} 사이클")
    print(f"예상 수명: {approval_result.predicted_years:.2f}년")
    print(f"요구 수명: {approval_result.required_years:.1f}년")
    print()
    
    if approval_result.meets_requirement:
        print("✓ 승인 기준 충족")
        margin = approval_result.predicted_years - approval_result.required_years
        print(f"  여유: +{margin:.2f}년 ({margin/approval_result.required_years*100:.1f}%)")
    else:
        print("✗ 승인 기준 미달")
        shortage = approval_result.required_years - approval_result.predicted_years
        print(f"  부족: -{shortage:.2f}년 ({shortage/approval_result.required_years*100:.1f}%)")
else:
    print("모델 피팅 실패로 승인 수명 예측을 건너뜁니다.")
    approval_result = None

### 5.3 온도별 수명 비교

In [ ]:
if fitting_success:
    # 여러 온도에서의 수명 예측
    target_temps = [15, 20, 25, 30, 35, 40]  # °C
    temp_results = []
    
    for temp_c in target_temps:
        temp_predictor = ApprovalLifePredictor(
            test_temperature=273.15 + TEST_TEMPERATURE_C,
            target_temperature=273.15 + temp_c,
            cycles_per_year=365,
            required_years=8.0
        )
        
        result = temp_predictor.fit_and_predict(
            cycles=cycles,
            temperatures=temperatures,
            capacities=capacities,
            activation_energy=50000
        )
        
        temp_results.append({
            'temperature_C': temp_c,
            'acceleration_factor': result.acceleration_factor,
            'predicted_years': result.predicted_years,
            'meets_requirement': result.meets_requirement
        })
    
    # DataFrame 생성
    df_temp = pd.DataFrame(temp_results)
    
    # 시각화
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # 가속 계수
    ax1.plot(df_temp['temperature_C'], df_temp['acceleration_factor'], 
             'b-o', linewidth=2, markersize=8)
    ax1.set_xlabel('실사용 온도 (°C)', fontsize=12)
    ax1.set_ylabel('가속 계수', fontsize=12)
    ax1.set_title('온도별 가속 계수', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # 예상 수명
    colors = ['green' if x else 'red' for x in df_temp['meets_requirement']]
    bars = ax2.bar(df_temp['temperature_C'], df_temp['predicted_years'],
                   color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
    ax2.axhline(y=8, color='red', linestyle='--', linewidth=2, label='요구 수명 (8년)')
    ax2.set_xlabel('실사용 온도 (°C)', fontsize=12)
    ax2.set_ylabel('예상 수명 (년)', fontsize=12)
    ax2.set_title('온도별 예상 수명', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=11)
    ax2.grid(True, alpha=0.3, axis='y')
    
    # 값 표시
    for bar, years in zip(bars, df_temp['predicted_years']):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{years:.1f}년', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print("\n=== 온도별 수명 비교 ===")
    print(df_temp.to_string(index=False))
else:
    print("모델 피팅 실패로 온도별 수명 비교를 건너뜁니다.")

## 6. 분석 리포트 생성

### 6.1 요약 리포트

In [ ]:
# 분석 결과 요약
print("="*70)
print(" " * 20 + "배터리 분석 리포트")
print("="*70)

print("\n[1] 데이터 정보")
print("-" * 70)
print(f"  데이터 경로: {DATA_PATH}")
print(f"  충방전기 타입: {cycler_type.name}")
print(f"  정격 용량: {loader.rated_capacity:.1f} mAh")
print(f"  총 사이클 수: {len(df_cycle)}")
print(f"  사이클 범위: {df_cycle['cycle'].min()} ~ {df_cycle['cycle'].max()}")

print("\n[2] 성능 지표")
print("-" * 70)
print(f"  초기 방전 용량: {df_cycle['discharge_capacity'].iloc[0]:.2f} mAh")
print(f"  최종 방전 용량: {df_cycle['discharge_capacity'].iloc[-1]:.2f} mAh")
print(f"  용량 유지율: {retention[-1]*100:.2f}%")
print(f"  용량 감소: {(1-retention[-1])*100:.2f}%")
if 'efficiency' in df_cycle.columns:
    print(f"  평균 효율: {df_cycle['efficiency'].mean():.2f}%")

if fitting_success:
    print("\n[3] 모델 피팅 결과")
    print("-" * 70)
    print(f"  RMSE: {result.rmse:.6f}")
    print(f"  R²: {result.r_squared:.6f}")
    print(f"  피팅 품질: {('우수' if result.r_squared > 0.99 else '양호' if result.r_squared > 0.95 else '보통')}")
    
    print("\n[4] 수명 예측 ({0}°C 시험 조건)".format(TEST_TEMPERATURE_C))
    print("-" * 70)
    print(f"  EOL 기준: 80% 잔존 용량")
    print(f"  예측 EOL 사이클: {eol_cycle:,} 사이클")
    
    if approval_result:
        print("\n[5] 승인 수명 예측 ({0}°C → {1}°C)".format(TEST_TEMPERATURE_C, TARGET_TEMPERATURE_C))
        print("-" * 70)
        print(f"  가속 계수: {approval_result.acceleration_factor:.2f}배")
        print(f"  실사용 환산 사이클: {approval_result.equivalent_real_cycles:,} 사이클")
        print(f"  예상 수명: {approval_result.predicted_years:.2f}년")
        print(f"  요구 수명: {approval_result.required_years:.1f}년")
        print(f"  승인 여부: {'✓ 통과' if approval_result.meets_requirement else '✗ 미달'}")
        
        if approval_result.meets_requirement:
            margin = approval_result.predicted_years - approval_result.required_years
            print(f"  여유: +{margin:.2f}년 ({margin/approval_result.required_years*100:.1f}%)")
        else:
            shortage = approval_result.required_years - approval_result.predicted_years
            print(f"  부족: -{shortage:.2f}년 ({shortage/approval_result.required_years*100:.1f}%)")

print("\n" + "="*70)
print(" " * 25 + "분석 완료")
print("="*70)

### 6.2 결과 DataFrame 생성

In [ ]:
# 결과를 DataFrame으로 정리
if fitting_success and approval_result:
    summary = {
        '항목': [
            '데이터 경로',
            '충방전기 타입',
            '정격 용량 (mAh)',
            '총 사이클 수',
            '초기 방전 용량 (mAh)',
            '최종 방전 용량 (mAh)',
            '용량 유지율 (%)',
            '모델 R²',
            '모델 RMSE',
            'EOL 사이클 (시험)',
            '가속 계수',
            'EOL 사이클 (실사용)',
            '예상 수명 (년)',
            '승인 여부'
        ],
        '값': [
            str(DATA_PATH),
            cycler_type.name,
            f"{loader.rated_capacity:.1f}",
            len(df_cycle),
            f"{df_cycle['discharge_capacity'].iloc[0]:.2f}",
            f"{df_cycle['discharge_capacity'].iloc[-1]:.2f}",
            f"{retention[-1]*100:.2f}",
            f"{result.r_squared:.6f}",
            f"{result.rmse:.6f}",
            f"{eol_cycle:,}",
            f"{approval_result.acceleration_factor:.2f}",
            f"{approval_result.equivalent_real_cycles:,}",
            f"{approval_result.predicted_years:.2f}",
            '통과' if approval_result.meets_requirement else '미달'
        ]
    }
    
    df_summary = pd.DataFrame(summary)
    
    print("\n=== 분석 결과 요약 테이블 ===")
    display(df_summary)
    
    # CSV로 저장 (옵션)
    # df_summary.to_csv('battery_analysis_report.csv', index=False, encoding='utf-8-sig')
    # print("\n✓ 리포트를 battery_analysis_report.csv로 저장했습니다.")
else:
    print("모델 피팅 또는 승인 수명 예측 실패로 요약 테이블을 생성할 수 없습니다.")

## 완료

배터리 데이터 분석 워크플로우가 완료되었습니다!

### 다음 단계

1. **추가 분석**: dV/dQ 분석, 내부 저항 분석 등
2. **다중 셀 비교**: 여러 셀의 데이터를 로드하여 비교 분석
3. **리포트 저장**: 결과를 CSV/Excel로 내보내기
4. **자동화**: 이 워크플로우를 Python 스크립트로 변환하여 배치 처리

### 참고

- 모듈 문서: `battery_analysis_v2` 모듈의 각 클래스/함수 docstring 참조
- 예제 노트북: `battery_analysis_v2_examples.ipynb` 참조